In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential as Base_Model
from tensorflow.keras.applications import VGG19 as Pretrained_Model
from tensorflow.keras.datasets import mnist as dataset
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
def create_model():

    pretrained_model = Pretrained_Model(
        input_shape=(224, 224, 3),
        include_top = False,
        weights='imagenet'
    )

    for layer in pretrained_model.layers:
        layer.trainable = False

    model = Base_Model([
        Input(shape=(28, 28, 1)),
        UpSampling3D(size=(8, 8, 3)),
        pretrained_model,
        Dropout(0.5),
        Flatten(),
        Dense(4096, activation='relu'),
        Dense(2048, activation='relu'),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax'),
    ])

    model.summary()

    return model

In [ ]:
# Load MNIST datasets
(x_train, y_train), (x_test, y_test) = dataset.load_data()
x_train, x_test = np.expand_dims(x_train, axis = 3), np.expand_dims(x_test, axis = 3)  

# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = np.array(x_train, dtype=np.float32)/255.0, np.array(x_test, dtype=np.float32)/255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10, dtype=np.uint8),tf.keras.utils.to_categorical(y_test, 10, dtype=np.uint8)

# Brief information of datasets
print("Shape of original training examples:", np.shape(x_train))
print("Shape of original test examples:", np.shape(x_test))
print("Shape of original training result:", np.shape(y_train))
print("Shape of original test result:", np.shape(y_test))

Shape of original training examples: (60000, 28, 28, 1)
Shape of original test examples: (10000, 28, 28, 1)
Shape of original training result: (60000, 10)
Shape of original test result: (10000, 10)


In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
    model = create_model()

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x_train, y_train, 
        validation_data = (x_test, y_test),
        batch_size=1024, 
        epochs=50,
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)]
    )

INFO:tensorflow:Initializing the TPU system: grpc://10.119.23.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.119.23.170:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


80142336/80134624 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling3d_1 (UpSampling3 (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dense_8 (Dense)              (None, 2048)              8390656   
_______________________________________________________

In [ ]:
model.evaluate(x_test, y_test, 1024)

10/10 [==============================] - 3s 278ms/step - loss: 0.0445 - accuracy: 0.9860


[0.04450763389468193, 0.9860000014305115]